In [37]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import re
import os
import pdfplumber


## scrping av del 1

In [38]:
from bs4 import BeautifulSoup, NavigableString
import requests

def extract_text(url):
    # Get the HTML of the page
    response = requests.get(url)
    html = response.text

    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Find the divs with the specified classes
    divs = soup.find_all(class_=["RichtextArea ProductPage__richtext text-wrapper", "layout--2col wrapper"])

    # Extract the text of each child of each div until a h2, h3 or ul tag is encountered
    text = []
    for div in divs:
        for child in div.children:
            if isinstance(child, NavigableString):
                text.append(child.strip())
            elif child.name in ['ul']:
                break
            else:
                text.append(child.get_text(strip=True))
    text_string = ' '.join(text)
        
    return text_string
df = pd.read_csv("data/tags_30_11.csv")
number = 23

print(extract_text(df["url"][number]))

print(df["url"][number])

 EK80 Portable EntryThe EK80 Portable Entry is all you need for ecosystem monitoring and analysis in one package – at a reasonable price.  EK80 Portable Entry bundle comprises:   This is a complete rugged echo sounder with a built-in computer, GPS, and WiFi for remote operation. The core of the product is the WBT Mini, a compact EK80 wideband transceiver with four channels. The Entry bundle uses three of these channels for the 38 kHz split beam, while the last channel is used for the 200 kHz single beam.    The EK80 Portable entry is provided with the ES38-18/200-18 transducer. This is a dual-frequency transducer combining 38 and 200 kHz in one housing. The beamwidth is 18 degrees at both frequencies to secure a large sampling volume in shallow water. Combining high and low frequencies is ideal for mapping ecosystem components and separating fish from plankton.   Key features 
https://www.kongsberg.com/maritime/products/ocean-science/ocean-science/es_scientific/sci_es_transceivers/ek80

## scraping av data sheets

In [39]:
def find_datasheets(url):
    # Get the HTML of the page
    response = requests.get(url)
    html = response.text

    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Find the div with the class "Downloads"
    downloads_div = soup.find('div', class_="Downloads")
    # downloads_span = soup.find('span', class_="Downloads_itemName")

    # If the div is found, find the section with the subtitle "Data sheet", "DATA SHEET", or "Data- and product sheets" within this div
    if downloads_div:
        # datasheets_subtitle = downloads_div.find('h3', class_="Section__subtitle", string=re.compile("data sheet|datasheet|data- and product sheets|Brochure", re.I))
        datasheets_itemname = downloads_div.find('span', class_="Downloads__itemName", string=re.compile("data sheet|Datasheet|data- and product sheets|Brochure", re.I))
        # # If the subtitle is found, find the next sibling section
        # if datasheets_subtitle:
        #     datasheets_section = datasheets_subtitle.find_next_sibling()

        #     # If the section is found, find all links within this section
        #     if datasheets_section:
        #         datasheet_links = datasheets_section.find_all('a', class_="Downloads__itemLink")

        #         # If the links are found, prepend "https://www.kongsberg.com" to each href attribute and return the list
        #         if datasheet_links:
        #             links = [link.get('href') if 'https://simrad.online' in link.get('href') or 'https://www.simrad.online' in link.get('href') else "https://www.kongsberg.com" + link.get('href') for link in datasheet_links if link.get('href').endswith('.pdf')]
        #             return ', '.join(links)        
        # if downloads_div:
        datasheets_subtitle = downloads_div.find('h3', class_="Section__subtitle", string=re.compile("data sheet|data- and product sheets|Brochure", re.I))

        datasheets_subtitle = downloads_div.find('h3', class_="Section__subtitle", string=re.compile("data sheet|datasheet|data- and product sheets|Brochure", re.I))
        datasheets_itemname = downloads_div.find('span', class_="Downloads__itemName", string=re.compile("data sheet|Datasheet|data- and product sheets|Brochure", re.I))
                    return ', '.join(links)
                    return ', '.join(links)        
        elif datasheets_itemname:
            datasheets_section = datasheets_itemname.find_parent().find_parent()

            if datasheets_section:
                datasheet_links = datasheets_section.find_all('a', class_="Downloads__itemLink")
                datasheet_links_with_datasheet = [link for link in datasheet_links if link.get_text(strip=True) == "Datasheet"]
                if datasheet_links_with_datasheet:
                    return [link.get('href') if 'https://simrad.online' in link.get('href') or 'https://www.simrad.online' in link.get('href') else "https://www.kongsberg.com" + link.get('href') for link in datasheet_links_with_datasheet if link.get('href').endswith('.pdf')]
    else:
        return None

In [48]:
def find_downloads(url):
    response = requests.get(url)
    html = response.text

    # Parse the HTML
    soup = BeautifulSoup(html, 'html.parser')

    # Dictionary to store the result
    section_links = {}

    # Find all section subtitles and corresponding links
    for subtitle in soup.find_all("h3", class_="Section__subtitle"):
        section_name = subtitle.text.strip()
        links = [a['href'] for a in subtitle.find_next_sibling("ul").find_all("a", href=True)]
        section_links[section_name] = links

    return section_links

In [49]:
df = pd.read_excel("data/12_04/tags_12_04.xlsx")

df["downloads"] = df["url"].apply(find_downloads)


,Product_Name,product_tags,Product category,url,image_url,downloads
0,Acoustic control system for BOP operation,"Acoustic Control System, Leak Detection System...",Surveillance & monitoring,https://www.kongsberg.com/maritime/products/Ac...,https://www.kongsberg.com/contentassets/4b9571...,{'Data sheet': ['/globalassets/maritime/km-pro...
1,AIS physical shore stations,"AIS transponder, acoustic telemetry link, mari...",Surveillance & monitoring,https://www.kongsberg.com/maritime/products/on...,https://www.kongsberg.com/contentassets/c8180d...,{'Brochure': ['~/link/349428ca632f4f61922083be...
2,AIS service management system,"Acoustic Control System, AIS transponder, remo...",Surveillance & monitoring,https://www.kongsberg.com/maritime/products/on...,https://www.kongsberg.com/contentassets/f3f5d1...,{}
3,AIS space receivers,"radio, transceiver, reconfigurable SDR, VDES t...",Communication solutions,https://www.kongsberg.com/maritime/products/on...,https://www.kongsberg.com/contentassets/b015aa...,{'Data sheets': ['/contentassets/6a9f1eb8cd564...
4,Analytics,"Data Management, Cloud Storage, cloud-based, A...",Cloud solution,https://www.kongsberg.com/maritime/products/oc...,https://www.kongsberg.com/contentassets/b26de6...,{'DATA SHEET': ['/contentassets/90db5b261a1543...
...,...,...,...,...,...,...
224,SA9510S,NaN,NaN,https://www.kongsberg.com/globalassets/maritim...,NaN,{}
225,SA9520,NaN,NaN,https://www.kongsberg.com/globalassets/maritim...,NaN,{}
226,SS2030,NaN,NaN,https://www.kongsberg.com/globalassets/maritim...,NaN,{}
227,ST2400,NaN,NaN,https://www.kongsberg.com/globalassets/maritim...,NaN,{}


In [52]:
df = df[['url', 'Product_Name', "downloads"]]

df.to_excel("data/downloads_files.xlsx", index=False)

In [24]:
df = pd.read_excel("data/12_04/tags_12_04.xlsx")

# Add a new column "Data sheets" to the dataframe
df['Data sheets'] = df['url'].apply(find_datasheets)
df["downloads"] = df["url"].apply(find_downloads)

# only keep column url, product name and data sheets
df = df[['url', 'Product_Name', 'Data sheets', "downloads"]]
df.to_csv("data/data_sheets_temp_4.csv", index=False)
df.to_excel("data/data_sheets_temp_4.xlsx", index=False)

## Scraping av PDF

In [14]:
import requests
from PyPDF2 import PdfReader
import io

def scrape_pdf_text(url):
    # Download the PDF file
    response = requests.get(url)
    with io.BytesIO(response.content) as f:
        # Open the PDF file
        reader = PdfReader(f)

        # Extract text from each page
        text = ''
        for page in reader.pages:
            text += page.extract_text()

    return text

# Example usage
pdf_url = 'https://www.kongsberg.com/globalassets/maritime/km-products/product-documents/hugin-superior.pdf'
pdf_text = scrape_pdf_text(pdf_url) 
print(pdf_text)

KeyboardInterrupt: 

In [ ]:
import requests
from PyPDF2 import PdfReader
import io

def scrape_pdf_text(url):
    # Download the PDF file
    response = requests.get(url)
    with io.BytesIO(response.content) as f:
        # Open the PDF file
        reader = PdfReader(f)

        # Extract text from each page
        text = ''
        for i in range(len(reader.pages)):
            page = reader.getPage(i)
            text += page.extractText()

    return text

# Example usage
pdf_url = 'https://www.kongsberg.com/globalassets/maritime/km-products/product-documents/hugin-superior.pdf'
pdf_text = scrape_pdf_text(pdf_url)
print(pdf_text)

DeprecationError: reader.getPage(pageNumber) is deprecated and was removed in PyPDF2 3.0.0. Use reader.pages[page_number] instead.

In [ ]:
def scrape_pdf_text(url):
    # Download the PDF file
    response = requests.get(url)
    with open('temp.pdf', 'wb') as f:
        f.write(response.content)

    # Open the PDF file
    with pdfplumber.open('temp.pdf') as pdf:
        # Extract text from each page
        text = ''
        in_features_section = False
        if len(pdf.pages) > 6:
            return None
        for i, page in enumerate(pdf.pages):
            # If this is the last page, define a crop box that excludes the last 1 cm from the bottom
            if i == len(pdf.pages) - 1:
                crop_box = (0, 0, page.width, page.height - 70)
                page = page.crop(crop_box)

            page_text = page.extract_text().split('\n')
                        
            for line in page_text:
                if re.search('features|FUNCTIONALITY', line, re.IGNORECASE):
                    in_features_section = True
                elif re.search('technical (specifications|highlights|data)', line, re.IGNORECASE):                    
                    break

                if not in_features_section:
                    line = line.replace('®', '')  # Remove ® symbol
                    if line.startswith('•'):
                        text += '\n' + line  # Add bullet point to new line
                    else:
                        text += line + '\n'

    # Remove the temporary PDF file
    os.remove('temp.pdf')

    return text

# Example usage
data_sheets = pd.read_csv("data/data_sheets.csv")
data_sheets = data_sheets[data_sheets['Data sheets'].notna()]
data_sheets = data_sheets[data_sheets['Data sheets'].str.count(',') < 1]
product_datasheet = data_sheets["Data sheets"].iloc[5]
pdf_text = scrape_pdf_text(product_datasheet)
print(pdf_text)
print(product_datasheet)




https://www.kongsberg.com/globalassets/maritime/km-products/product-documents/hugin-superior.pdf


In [ ]:

# Load the CSV file
data_sheets = pd.read_csv("data/data_sheets.csv")
data_sheets = data_sheets.head(40)

# Filter rows with valid data sheet URLs
data_sheets = data_sheets[data_sheets['Data sheets'].notna()]
data_sheets = data_sheets[data_sheets['Data sheets'].str.count(',') < 1]

# Scrape the text from each data sheet and save it as a new column
data_sheets['Scraped Text'] = data_sheets['Data sheets'].apply(scrape_pdf_text)

# Save the DataFrame to a new CSV file
data_sheets.to_csv("data/data_sheets_with_text.csv", index=False)

## GPT formating

In [ ]:
import streamlit as st
import pandas as pd
from openai._client import OpenAI

client = OpenAI(
    api_key=st.secrets["openai"]["api_key"],
)


In [ ]:
def generate_text(text):
    messages = [
            {
                "role": "system",
                "content": "You are a text editor, that edits text into readable text bulks and headings. Use the same kind of language and tone and write style as the text you are editing. Do not change the meaning of the text, or add any new information. Format the text as little as possible."
            },
            {
                "role": "user",
                "content": f"I have a text that I want to format into headings and text bulks. The text is:\n\n{text}\n\nPlease format this text."
            }
        ]

    response = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=messages,
    )
    output_text = response.choices[0].message.content.strip()
    return output_text 

In [ ]:
df_with_text = pd.read_csv("data/data_sheets_with_text.csv")

input_text = (df_with_text["Scraped Text"][2])
output_text = generate_text(input_text)

print(output_text)

KONGSBERG AUTONOMOUS UNDERWATER VEHICLES

**HUGIN AUV System**

HUGIN is the most successful commercial Autonomous Underwater Vehicle (AUV) available. It combines IHO quality positioning with the


In [ ]:
print(input_text)

BLUE INSIGHT
The digital ocean toolbox
Seamless data acquisition, fusion, processing, visualization,
contextualization and management of ocean data.
Kongsberg Blue Insight is an open and flexible digital infrastructure
product designed to meet the increasing demand for accurate information
from the oceans, focusing on internet security and seamless integration.
The Blue Insight automates the data stream from a wide range of sensors
KEY BENEFITS: and platform types and visualizes them in a common framework. The
solution facilitates modern data processing and machine learning, provid-

• One-stop solution that ing new understanding and in-depth knowledge.provides real time
overview across platforms Through a range of dedicated modules, Blue Insight offers flexibil-
and data types
ity, security and robustness required by any ocean data stakeholder:

• Flexible and scalable,scientific, observational, or commercial.
suitable for both small
and large organizations

• Open ecosystem buildingo